In [0]:
import mlflow.sklearn
import mlflow
import mlflow.pyfunc
import pandas as pd
# from sklearn.datasets import make_regression

In [0]:
# imports
import pandas as pd
import numpy as np
import pytz
from datetime import datetime, timedelta, time
from prophet import Prophet
from prophet.make_holidays import make_holidays_df
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from pyspark.sql.functions import to_timestamp
from prophet.plot import plot_forecast_component
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, StructType, DoubleType, LongType
from pyspark.ml.feature import VectorAssembler, StandardScaler, StringIndexer, OneHotEncoder, MinMaxScaler
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, MultilayerPerceptronClassifier
from pyspark.mllib.evaluation import MulticlassMetrics,BinaryClassificationMetrics
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import functions as f
from pyspark.sql.window import Window
from pyspark.sql.functions import col, when, to_timestamp, lit, udf
from pyspark.ml import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt
from pyspark.sql.functions import col, to_timestamp, to_date, when
from prophet.make_holidays import make_holidays_df


In [0]:
team_BASE_DIR = f"dbfs:/student-groups/Group_4_1"

df = spark.read.parquet(f"{team_BASE_DIR}/interim/join_checkpoints/joined__timefeat_seasfeat_cleaned_pr_v2.parquet")


## Prior Flight EDA 

In [0]:

team_BASE_DIR = f"dbfs:/student-groups/Group_4_1"

df = spark.read.parquet(f"{team_BASE_DIR}/interim/join_checkpoints/joined__timefeat_seasfeat_cleaned_pr_v2.parquet")


In [0]:
df = df.withColumn("outcome", (when((col("DEP_DELAY") >= 15) | (col("CANCELLED") == 1), 1).otherwise(0)).cast("double"))

In [0]:
df.columns

In [0]:
priorflight_cols = ['priorflight_cancelled_true',
 'priorflight_origin',
 'priorflight_dest',
 'priorflight_sched_deptime',
 'priorflight_true_deptime',
 'priorflight_sched_elapsed',
 'priorflight_true_elapsed',
 'priorflight_true_depdelay',
 'priorflight_sched_arrtime',
 'priorflight_true_arrtime',
 'priorflight_depdelay_calc',
 'priorflight_isdeparted',
 'priorflight_deptime_calc',
 'priorflight_isdelayed_calc',
 'priorflight_isarrived_calc',
 'priorflight_arr_time_calc',
 'turnaround_time_calc']

display(df.select(priorflight_cols).describe())

In [0]:
turnaround_array = df.select("turnaround_time_calc").dropna().rdd.flatMap(lambda x: x).collect()
plt.figure(figsize=(8, 5))
plt.hist(turnaround_array, bins=30, color='skyblue', edgecolor='black')
plt.title("Turnaround Time Histogram")
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.grid(True)
plt.show()

In [0]:
groups

In [0]:
# pdf = df.select("turnaround_time_calc", "outcome").dropna().sample(False, 0.05).toPandas()
pdf = df.select("turnaround_time_calc", "outcome").dropna().toPandas()

# Step 2: Prepare data for stacking
# groups = pdf["outcome"].unique()
groups = [0,1]
data = [pdf[pdf["outcome"] == g]["turnaround_time_calc"].values for g in groups]

# Step 3: Plot stacked histogram
plt.figure(figsize=(10, 6))
plt.hist(data, bins=30, stacked=True, label=groups, color= ['skyblue','lightcoral'],edgecolor='dimgrey', range=(-400,1000))
plt.xlabel("Turnaround Time in Minutes")
plt.ylabel("Frequency")
plt.title("Aircraft Turnaround Time Histogram")
plt.legend(title="Flight Delayed")
plt.ticklabel_format(scilimits=(-5, 8))
# plt.grid(True)
plt.tight_layout()
plt.show()

# RF 1

In [0]:
# convert time variable to datetime
df = df.withColumn('sched_depart_utc', to_timestamp(col('sched_depart_utc')))

# add hour and date variables (needed for seasonality and CV splits, respectively)
df = df.withColumn("dep_hour_utc", f.hour(col('sched_depart_utc'))) \
    .withColumn("dep_date_utc", to_date(col('sched_depart_utc')))

# define outcome variable
df = df.withColumn("outcome", (when((col("DEP_DELAY") >= 15) | (col("CANCELLED") == 1), 1).otherwise(0)).cast("double"))

# cast weather columns to double
weather_cols = [col for col in df.columns if "origin_Hourly" in col]
remove_me = ["origin_HourlyPresentWeatherType","origin_HourlySkyConditions","origin_HourlyWindDirection"]
num_weather_cols = [c for c in weather_cols if c not in remove_me]
for column in num_weather_cols:
    df = df.withColumn(column, col(column).cast("double"))

df.cache()


In [0]:
df = df.withColumn('pagerank_train', (df['train_0'] + df['train_1'] + df['train_2'] + df['train_3'] + df['train_4']) / 5)

min_test_dt = "2019-01-01"

# split into train and test
df_train = df.filter(f.col('sched_depart_utc') < min_test_dt)
# df_train.cache()
# print(f"Train data: {df_train.count()} records")
df_test = df.filter(f.col('sched_depart_utc') >= min_test_dt) \
    .filter(f.col('sched_depart_utc') < "2020-01-01")
# df_test.cache()
# print(f"Test data: {df_test.count()} records")

In [0]:
# https://dbc-fae72cab-cf59.cloud.databricks.com/ml/experiments/de11694479cf4eb689ea73f21bc9a0d9/runs/a6ee34f59a564e028e08df1e2eb5fa27?o=4021782157704243
# glamorous-whale-645
# RF model 
run_id = "a6ee34f59a564e028e08df1e2eb5fa27"
model_uri = f"runs:/{run_id}/model"
loaded_model = mlflow.pyfunc.load_model(model_uri)

In [0]:
for stage in loaded_model.stages:
    if stage.__class__.__name__ == "VectorAssembler":
        feature_cols = stage.getInputCols()
        break
print(len(feature_cols))
feature_cols

In [0]:
importances = loaded_model.stages[-1].featureImportances
print(len(importances))
importances

In [0]:
importances.values

In [0]:
fi_df = pd.DataFrame({
    'feature': feature_cols,
    'importance': importances.toArray()
}).sort_values('importance', ascending=False)

In [0]:
loaded_model.stages[-3].transform

In [0]:
# # unwrapped_model = loaded_model.unwrap_python_model()

# # data = {
# #     # 'feature_name': unwrapped_model.model.feature_name_,
# #     'imp': unwrapped_model.model.feature_importances_
# # }

# # fi = pd.DataFrame(data).sort_values(by='imp', ascending=False)
# # top_10_features = fi.head(10).to_list()
# # # display(top_10_features)
# # unwrapped_model = loaded_model.unwrap_python_model()

# # data = {
# #     # 'feature_name': unwrapped_model.model.feature_name_,
# #     'imp': unwrapped_model.model.feature_importances_
# # }

# # fi = pd.DataFrame(data).sort_values(by='imp', ascending=False)
# # top_10_features = fi.head(10)['feature_name'].to_list()
# # display(top_10_features)
# feature_metadata = transformed_df.schema[assembler.getOutputCol()].metadata

# # Reconstruct feature names
# from pyspark.ml.attribute import AttributeGroup

# attrs = AttributeGroup.fromStructField(transformed_df.schema[assembler.getOutputCol()])
# expanded_feature_names = [attr.name if attr.name else f"feature_{i}" for i, attr in enumerate(attrs.attributes)]

In [0]:
transformed_test = loaded_model.transform(df_test.withColumnsRenamed({'daily_full':'daily', 'weekly_full':'weekly', 'yearly_full':'yearly', 'holidays_full':'holidays', 'pagerank_train':'pagerank'}))


In [0]:
loaded_model.stages[-3].getInputCols()

In [0]:
loaded_model.stages[-1].extractParamMap()

In [0]:
loaded_model.stages[-1].toDebugString

In [0]:
feats=['origin_HourlyDewPointTemperature', 'origin_HourlyDryBulbTemperature', 'origin_HourlyPrecipitation', 'origin_HourlyPressureChange', 'origin_HourlyRelativeHumidity', 'origin_HourlyVisibility', 'origin_HourlyWetBulbTemperature', 'origin_HourlyWindGustSpeed', 'origin_HourlyWindSpeed', 'daily', 'weekly', 'yearly', 'holidays', 'mean_dep_delay', 'prop_delayed', 'turnaround_time_calc', 'priorflight_depdelay_calc', 'DISTANCE', 'CRS_ELAPSED_TIME', 'pagerank', 'YEAR_vec', 'MONTH_vec', 'DAY_OF_MONTH_vec', 'DAY_OF_WEEK_vec', 'OP_UNIQUE_CARRIER_vec', 'ORIGIN_ICAO_vec', 'DEST_ICAO_vec', 'priorflight_isdeparted_vec', 'priorflight_isarrived_calc_vec', 'priorflight_isdelayed_calc_vec', 'priorflight_cancelled_true_vec']

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator



In [0]:
evaluator = MulticlassClassificationEvaluator(
        labelCol="outcome", 
        metricName="fMeasureByLabel",
        beta=2.0,
        metricLabel=1.0
      )

score = evaluator.evaluate(transformed_test)

In [0]:
va = loaded_model.stages[-3]
tree = loaded_model.stages[-1]


list(zip(va.getInputCols(), tree.featureImportances))

In [0]:
pd.DataFrame(list(zip(va.getInputCols(), tree.featureImportances))).sort_values(by=1, ascending=False).rename({0:'feature', 1:'importance'}, axis=1).reset_index(drop=True)

In [0]:
dfx = pd.DataFrame(list(zip(va.getInputCols(), tree.featureImportances))).sort_values(by=1, ascending=False)

In [0]:
dfx

In [0]:
dfx10 = dfx[:10]

In [0]:
plt.figure(figsize=(8, 4))
plt.barh(dfx10[0], dfx10[1], color='skyblue')
plt.xlabel('Feature Importance')
plt.title('Random Forest Feature Importance')

plt.gca().invert_yaxis()  # Invert y-axis for better visualization
plt.show()

In [0]:
display(transformed_test)

In [0]:
transformed_test.columns

In [0]:
t

# Random forest - hyperopted

featureSubsetStrategy: auto

maxDepth: 10

model: rf

numTrees: 60